In [1]:
import os
import shutil
import pandas as pd
import cv2

In [2]:
base_path = "/home/aezzakri/Dvlp/py_test/drivdata/"
source_dir = 'images/'
destination_dir = 'dataset/'
data_dir = base_path + "data/"
for dir in os.listdir(data_dir):
    if dir != "no_background" and dir != "f":
        print(data_dir + dir)

/home/aezzakri/Dvlp/py_test/drivdata/data/0
/home/aezzakri/Dvlp/py_test/drivdata/data/5
/home/aezzakri/Dvlp/py_test/drivdata/data/b
/home/aezzakri/Dvlp/py_test/drivdata/data/1
/home/aezzakri/Dvlp/py_test/drivdata/data/2
/home/aezzakri/Dvlp/py_test/drivdata/data/8
/home/aezzakri/Dvlp/py_test/drivdata/data/6
/home/aezzakri/Dvlp/py_test/drivdata/data/a
/home/aezzakri/Dvlp/py_test/drivdata/data/4
/home/aezzakri/Dvlp/py_test/drivdata/data/d
/home/aezzakri/Dvlp/py_test/drivdata/data/c
/home/aezzakri/Dvlp/py_test/drivdata/data/3
/home/aezzakri/Dvlp/py_test/drivdata/data/e
/home/aezzakri/Dvlp/py_test/drivdata/data/7
/home/aezzakri/Dvlp/py_test/drivdata/data/9


In [3]:
labels = pd.read_csv('../train_labels.csv')

In [4]:
source_dir = '/home/aezzakri/Dvlp/py_test/drivdata/data/1/images/'
destination_dir = '/home/aezzakri/Dvlp/py_test/drivdata/dataset/'
nb_images = len(os.listdir(source_dir))
counter = 0

In [5]:
nb_train = int(nb_images*1)
nb_val = int(nb_images*0)

train_imgs, val_imgs = os.listdir(source_dir)[:nb_train], os.listdir(source_dir)[nb_train:nb_train+nb_val]

In [6]:
def copy_img(image, dest_dir):
    source_path = os.path.join(source_dir, image)
    destination_path = os.path.join(dest_dir, image)
    shutil.copy(source_path, destination_path)
    return source_path, destination_path

In [7]:
def read_img(source_path):
    im = cv2.imread(source_path)
    h, w, c = im.shape
    return h, w, c

In [8]:
def make_directories_yolo():
    os.mkdir(destination_dir + 'images')
    os.mkdir(destination_dir + 'images/train')
    os.mkdir(destination_dir + 'images/val')

    os.mkdir(destination_dir + 'labels')
    os.mkdir(destination_dir + 'labels/train')
    os.mkdir(destination_dir + 'labels/val')

In [12]:
# make_directories_yolo()

In [64]:
# rows = labels.iterrows()

In [10]:

def make_data_yolo(mode, img_list):
    imgs_dir = destination_dir + f'/images/{mode}'
    labels_dir = destination_dir + f'/labels/{mode}'
    compteur = 0
    for image in img_list:
        try:
            print(compteur, end=" ")
            compteur += 1
            source_path, destination_path = copy_img(image, imgs_dir)
            print(os.path.join(imgs_dir, image))
            h, w, c = read_img(os.path.join(imgs_dir, image))
            image_name, _ = os.path.splitext(image)
        except Exception:
            print("error")

        try:

            for _, row in labels.iterrows():
                image_id = row['image_id']
                if image_id == image_name:
                    xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
                    # Calculate bounding box center and normalized coordinates
                    x_center = (int(xmin) + int(xmax)) / 2
                    y_center = (int(ymin) + int(ymax)) / 2
                    width_normalized = (int(xmax) - int(xmin)) / w
                    height_normalized = (int(ymax) - int(ymin)) / h

                    line = f"0 {x_center / w:.6f} {y_center / h:.6f} {width_normalized:.6f} {height_normalized:.6f}"

                    with open(labels_dir + f"/{image_name}.txt", 'w') as output_file:
                        # Write each element of the list (row) with comma separation and newline
                        output_file.write(line)
        except Exception:
            print("eeror")

        # except KeyError:
        #     print(f"Error: {image_name} column not found in DataFrame.")
    print("\nFinished")


In [ ]:

destination_dir = '/home/aezzakri/Dvlp/py_test/drivdata/dataset/'

counter = 0

base_path = "/home/aezzakri/Dvlp/py_test/drivdata/"
source_dir = 'images/'

data_dir = base_path + "data/"
for dir in os.listdir(data_dir):
    if dir != "no_background" and dir != "f":
        source_dir = data_dir + dir + "/images/"
        nb_images = len(os.listdir(source_dir))
        print(source_dir)
        nb_train = int(nb_images*1)
        nb_val = int(nb_images*0)

        train_imgs, val_imgs = os.listdir(source_dir)[:nb_train], os.listdir(source_dir)[nb_train:nb_train+nb_val]

        print("Creating train set")
        make_data_yolo('train', train_imgs)

In [2]:
# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

# Model Building
import ultralytics
from ultralytics import YOLO
from wandb.integration.ultralytics import add_wandb_callback
import wandb
# import labelme2yolo

# Others
import os
import shutil
import zipfile

%matplotlib inline

/home/aezzakri/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
wandb.init(project="PP_yolo", job_type="training")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: ezzakrianas65. Use `wandb login --relogin` to force relogin


In [11]:
model = YOLO('yolov8n.pt')

In [16]:
add_wandb_callback(model, enable_model_checkpointing=True)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [ ]:
results = model.train(data='/home/aezzakri/Dvlp/py_test/drivdata/dataset_aug/dataset.yaml', epochs=800, imgsz=640, project='Results_aug_v8')

In [15]:
wandb.finish()

In [14]:
wandb.init(project="PP_yolo", job_type="training")

In [ ]:
model = YOLO('yolov9e.pt')

In [ ]:
add_wandb_callback(model, enable_model_checkpointing=True)

In [ ]:
results = model.train(data='/home/aezzakri/Dvlp/py_test/drivdata/dataset_aug/dataset.yaml', epochs=800, imgsz=640, project='Results_aug_v9e')

In [ ]:
wandb.finish()

In [ ]:
wandb.init(project="PP_yolo", job_type="training")

In [ ]:
model = YOLO('yolov9c.pt')

In [ ]:
add_wandb_callback(model, enable_model_checkpointing=True)

In [ ]:
results = model.train(data='/home/aezzakri/Dvlp/py_test/drivdata/dataset_aug/dataset.yaml', epochs=800, imgsz=640, project='Results_aug_v9c')

In [ ]:
model = YOLO('yolov9e.pt')

In [ ]:
add_wandb_callback(model, enable_model_checkpointing=True)

In [ ]:
results = model.train(data='/home/aezzakri/Dvlp/py_test/drivdata/dataset_aug/dataset.yaml', epochs=800, imgsz=640, project='Results_aug_v9e')

In [ ]:
wandb.finish()

In [ ]:
wandb.init(project="PP_yolo", job_type="training")

In [21]:
!python3 train.py --img-size 640 --cfg cfg/training/yolov7.yaml --hyp data/hyp.scratch.custom.yaml --batch 8 --epochs 500 --data /home/aezzakri/Dvlp/py_test/drivdata/dataset_aug/dataset.yaml --weights yolov7_training.pt --workers 24 --name yolo_v7_sate

python3: can't open file '/home/aezzakri/Dvlp/py_test/drivdata/notebook/train.py': [Errno 2] No such file or directory


In [25]:
!git status

On branch main
nothing to commit, working tree clean
